In [1]:
#загружаем векторные файлы по одному.
#режем на экземпляры для обучения
import os
import sys
sys.path.insert(1, '../env')
sys.path.insert(1, '../agents')
sys.path.insert(1, '../')

import random
import numpy as np
import keras

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import pandas as pd
import pickle
import copy

import tensorflow as tf
import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout
from keras.layers import GRU, LSTM
from keras.optimizers import Adam


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
#размерность (1001, 1280)
width = 1280
sample_input_len = 15
sample_output_len = 15

In [4]:
#[sample_count_train, sequence_len, width]
model = Sequential()
reg = keras.regularizers.L1L2(l1=0.001, l2=0.001)
lsize = 256
model.add(LSTM(lsize, input_shape=(sample_input_len, width), kernel_regularizer=reg,return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(rate=0.3))
lsize = 100
model.add(Dense(lsize))

lsize = width#1280
model.add(Dense(lsize))
#model.add(LSTM(lsize, kernel_regularizer=reg))
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

In [5]:
#список названий файлов

directory = './movies_save_embeddings/'
files = os.listdir(directory)
i = 0

samples_per_file = 100

X_list = []
Y_list = []
for file in files:
    print(i, len(files), pd.Timestamp.now())
    if (not ('README') in file) and not ('.ipynb_checkpoints' in file):
        embedding_package = pickle.load(open('./movies_save_embeddings/' + file,'rb'))
        for s in range(samples_per_file):
            #разрезать эмбеддинг на сэмплы
            sample_start = int(np.random.rand()*(embedding_package.shape[0] - 1 - sample_input_len - sample_output_len))

            sample_input = embedding_package[sample_start:sample_start + sample_input_len,:]
            sample_output = embedding_package[sample_start + sample_input_len:sample_start + sample_input_len + sample_output_len,:]
            X_list.append(sample_input)
            Y_list.append(sample_output)
        i += 1
X = np.array(X_list)
Y = np.array(Y_list)
del X_list
del Y_list
idx_border = int(2*np.shape(X)[0]/3)
train_X = X[:idx_border,:,:]
train_Y = Y[:idx_border,:,:]
test_X = X[idx_border:,:,:]
test_Y = Y[idx_border:,:,:]

0 14 2020-08-07 19:49:09.920795
1 14 2020-08-07 19:49:09.930374
2 14 2020-08-07 19:49:09.936225
3 14 2020-08-07 19:49:09.942973
4 14 2020-08-07 19:49:09.951236
5 14 2020-08-07 19:49:09.957042
6 14 2020-08-07 19:49:09.962226
7 14 2020-08-07 19:49:09.967441
8 14 2020-08-07 19:49:09.972913
9 14 2020-08-07 19:49:09.978368
10 14 2020-08-07 19:49:09.984537
11 14 2020-08-07 19:49:09.991334
12 14 2020-08-07 19:49:09.996711
13 14 2020-08-07 19:49:10.001728


In [6]:
history = model.fit(train_X, train_Y, epochs=1000, batch_size=128, validation_data=(test_X, test_Y), verbose=1, shuffle=False)

Epoch 1/1000
8/8 [==============================] - 1s 139ms/step - loss: 34.8048 - val_loss: 33.9112
Epoch 2/1000
8/8 [==============================] - 0s 49ms/step - loss: 33.7036 - val_loss: 32.8773
Epoch 3/1000
8/8 [==============================] - 0s 46ms/step - loss: 32.6459 - val_loss: 31.8534
Epoch 4/1000
8/8 [==============================] - 0s 45ms/step - loss: 31.6087 - val_loss: 30.8418
Epoch 5/1000
8/8 [==============================] - 0s 47ms/step - loss: 30.5908 - val_loss: 29.8440
Epoch 6/1000
8/8 [==============================] - 0s 45ms/step - loss: 29.5892 - val_loss: 28.8613
Epoch 7/1000
8/8 [==============================] - 0s 44ms/step - loss: 28.6049 - val_loss: 27.8941
Epoch 8/1000
8/8 [==============================] - 0s 46ms/step - loss: 27.6380 - val_loss: 26.9431
Epoch 9/1000
8/8 [==============================] - 0s 46ms/step - loss: 26.6881 - val_loss: 26.0089
Epoch 10/1000
8/8 [==============================] - 0s 46ms/step - loss: 25.7554 - val_lo

8/8 [==============================] - 0s 47ms/step - loss: 0.2049 - val_loss: 0.1602
Epoch 83/1000
8/8 [==============================] - 0s 44ms/step - loss: 0.3583 - val_loss: 0.2639
Epoch 84/1000
8/8 [==============================] - 0s 45ms/step - loss: 0.6449 - val_loss: 1.2871
Epoch 85/1000
8/8 [==============================] - 0s 45ms/step - loss: 0.4978 - val_loss: 0.5351
Epoch 86/1000
8/8 [==============================] - 0s 45ms/step - loss: 0.3820 - val_loss: 0.4093
Epoch 87/1000
8/8 [==============================] - 0s 45ms/step - loss: 0.3278 - val_loss: 0.3540
Epoch 88/1000
8/8 [==============================] - 0s 44ms/step - loss: 0.2788 - val_loss: 0.2738
Epoch 89/1000
8/8 [==============================] - 0s 49ms/step - loss: 0.2459 - val_loss: 0.2341
Epoch 90/1000
8/8 [==============================] - 0s 44ms/step - loss: 0.2220 - val_loss: 0.2114
Epoch 91/1000
8/8 [==============================] - 0s 45ms/step - loss: 0.2045 - val_loss: 0.1782
Epoch 92/1000


8/8 [==============================] - 0s 46ms/step - loss: 0.0908 - val_loss: 0.0640
Epoch 164/1000
8/8 [==============================] - 0s 50ms/step - loss: 0.0945 - val_loss: 0.0780
Epoch 165/1000
8/8 [==============================] - 0s 45ms/step - loss: 0.1008 - val_loss: 0.0998
Epoch 166/1000
8/8 [==============================] - 0s 46ms/step - loss: 0.1035 - val_loss: 0.0953
Epoch 167/1000
8/8 [==============================] - 0s 45ms/step - loss: 0.1236 - val_loss: 0.1104
Epoch 168/1000
8/8 [==============================] - 0s 46ms/step - loss: 0.1406 - val_loss: 0.2409
Epoch 169/1000
8/8 [==============================] - 0s 46ms/step - loss: 0.2139 - val_loss: 0.6586
Epoch 170/1000
8/8 [==============================] - 0s 45ms/step - loss: 0.3175 - val_loss: 0.7529
Epoch 171/1000
8/8 [==============================] - 0s 46ms/step - loss: 0.2764 - val_loss: 0.2694
Epoch 172/1000
8/8 [==============================] - 0s 45ms/step - loss: 0.2223 - val_loss: 0.1795
Epoch

KeyboardInterrupt: 

In [ ]:
np.shape(test_Y)

In [7]:
y_pred = model.predict(test_X[0:1,:,:])

In [8]:
np.shape(y_pred)

(1, 15, 1280)

In [9]:
#измерить ошибку
test_Y_pred = model.predict(test_X[:,:,:])
rootmse_rel = np.sqrt(np.mean((test_Y_pred - test_Y)**2)/np.mean((test_Y)**2))
rootmse_std = np.mean((test_Y_pred - test_Y)**2)/np.std((test_Y)**2)
print(rootmse_rel,rootmse_std)

0.2906084 0.026523167
